In [32]:
# import requests library to make API calls 
import requests

# import pandas 
import pandas as pd

In [53]:
# define a function to call Alpha Vantage API
def call_alphavantage(etf): 
    URL = 'https://www.alphavantage.co/query?'
    PARAMS = {'function': 'TIME_SERIES_DAILY', 
              'symbol': etf, 
              'apikey': 'QQWPF2KTW701PG7H',
              'outputsize': 'full'
             }
    response = requests.get(URL, params=PARAMS)
    print('status code:', response.status_code)
    df = pd.DataFrame(response.json()['Time Series (Daily)'])
    return df

status code: 200


,2019-10-04,2019-10-03,2019-10-02,2019-10-01,2019-09-30,2019-09-27,2019-09-26,2019-09-25,2019-09-24,2019-09-23,...,2004-12-02,2004-12-01,2004-11-30,2004-11-29,2004-11-26,2004-11-24,2004-11-23,2004-11-22,2004-11-19,2004-11-18
1. open,141.4800,141.3600,140.7900,138.0700,139.7700,140.3500,142.1900,144.0800,143.2100,143.2200,...,45.5500,45.2800,45.3700,45.1000,45.2500,44.9293,44.8800,44.7500,44.4900,44.4300
2. high,142.3100,143.2500,141.8800,140.1900,140.2100,141.6800,142.5700,144.2900,144.7700,143.9400,...,45.5700,45.5900,45.4100,45.5000,45.6000,45.0500,44.9200,44.9700,44.9200,44.4900
3. low,141.1400,141.2900,140.4600,137.8000,138.0300,140.1600,141.5600,141.4400,142.8800,143.0700,...,44.6800,45.2600,44.8200,45.0829,45.0600,44.7900,44.7200,44.7400,44.4700,44.0700
4. close,141.9000,141.9000,141.2600,139.6300,138.8700,141.0600,141.7900,141.8300,144.5100,143.7500,...,44.9500,45.3800,45.1200,45.4000,45.2900,45.0500,44.7500,44.9500,44.7800,44.3800
5. volume,9652594,12396100,11914300,12476200,15459100,9804200,7984600,11648400,10576300,9006500,...,6457800,2037500,3857200,3759000,3097700,6105100,3169200,11996000,11655300,5992000


In [46]:
# define a function that renames columns after calling the API and formatting as a df 
def call_api_and_rename(etf):
    df = call_alphavantage(etf)        
    df = df.T
    df.columns = ['open_{}'.format(etf), 
                  'high_{}'.format(etf),
                  'low_{}'.format(etf),
                  'close_{}'.format(etf),
                  'volume_{}'.format(etf)] 
    return df

In [47]:
# call function and set resulting df equal to variable 


df_gld = call_api_and_rename('GLD') # gold bullion
df_gdx = call_api_and_rename('GDX') # gold mining
df_slv = call_api_and_rename('SLV') # silver bullion
df_sil = call_api_and_rename('SIL') # silver mining
df_index = call_api_and_rename('DJI') # Dow Jones Industrial Average 


status code: 200
status code: 200
status code: 200
status code: 200


In [52]:
# join all five pandas dataframes together 
df_gld_gdx = df_gld.join(df_gdx, how='inner')
df_gld_gdx_slv = df_gld_gdx.join(df_slv, how='inner')
df_gld_gdx_slv_sil = df_gld_gdx_slv.join(df_sil, how='inner')
df_gld_gdx_slv_sil_dji = df_gld_gdx_slv_sil.join(df_sil, how='inner')
len(df_gld_gdx_slv_sil_dji)

2383

In [54]:
# view records for combined df 
df_gld_slv_cpr.head()

,open_GLD,high_GLD,low_GLD,close_GLD,volume_GLD,open_SLV,high_SLV,low_SLV,close_SLV,volume_SLV,open_PPLT,high_PPLT,low_PPLT,close_PPLT,volume_PPLT,open_COPX,high_COPX,low_COPX,close_COPX,volume_COPX
2019-10-04,141.4800,142.3100,141.1400,141.9000,9652594,16.3000,16.4600,16.2500,16.4100,17178596,82.7700,83.3300,82.4000,83.1500,85272,16.3500,16.4999,16.3400,16.4400,21008
2019-10-03,141.3600,143.2500,141.2900,141.9000,12396100,16.3800,16.6500,16.3700,16.4300,24412700,84.1900,84.7900,83.7000,83.9700,106500,16.2700,16.4700,16.1800,16.4600,16000
2019-10-02,140.7900,141.8800,140.4600,141.2600,11914300,16.3300,16.5400,16.2700,16.4300,21300800,83.6400,84.1800,83.4000,83.8000,102300,16.5900,16.6200,16.3100,16.3500,40300
2019-10-01,138.0700,140.1900,137.8000,139.6300,12476200,15.9600,16.2100,15.9100,16.1500,23278100,82.7900,83.2600,82.5000,82.9200,97900,17.1000,17.1000,16.6400,16.7500,18500
2019-09-30,139.7700,140.2100,138.0300,138.8700,15459100,16.0300,16.1000,15.8300,15.9200,29218300,84.6600,85.2700,82.5000,83.4700,417600,17.2800,17.3300,17.1100,17.1100,12600
